In [2]:
import os
os.chdir('..')

In [ ]:
import numpy as np
import pathlib
import pickle
import pandas as pd

# Analysis of a SF graph

In [ ]:
results = np.empty((0, 3))
for file_path in pathlib.Path('results').glob('botnet_SF_N10000_*'):

    bB = float(file_path.stem.split('_bB')[1].split('_')[0])
    bW = float(file_path.stem.split('_bW')[1].split('_')[0])

    with open(file_path, 'rb') as file:
        data_list = pickle.load(file)

    botnet = [botnet for botnet, protected in data_list if protected > 0.01]
    botnet = np.mean(botnet) if len(botnet) else 1

    results = np.vstack((results,
                         np.array([bB, bW, botnet])))

results = results[results[:, 1].argsort()]
results = results[results[:, 0].argsort(kind='mergesort')]

data = pd.DataFrame({'beta_b': results[:, 0], 'beta_w': results[:, 1], 'botnet': results[:, 2]})
data.to_csv('plots/SF_botnet_size.csv', index=False)

In [ ]:
# Figure 4B
results = np.empty((0, 4))
for file_path in pathlib.Path('results').glob('protected_SF_N10000_*'):

    bW = float(file_path.stem.split('_bW')[1].split('_')[0])
    e = float(file_path.stem.split('_e')[1].split('_')[0])

    if bW in [2.0, 5.0, 8.0]:
        with open(file_path, 'rb') as file:
            data_list = pickle.load(file)

        self = [self for self, forced in data_list if (self + forced) > 0.01]
        self = np.mean(self) if len(self) else 0

        protected = [self + forced for self, forced in data_list if (self + forced) > 0.01]
        protected = np.median(protected) if len(protected) else 0

        results = np.vstack((results,
                             np.array([bW, e, self, protected])))

results = results[results[:, 1].argsort()]
results = results[results[:, 0].argsort(kind='mergesort')]

data = pd.DataFrame({'beta': results[:, 0], 'epsilon': results[:, 1], 
                     'self': results[:, 2], 'protected': results[:, 3]})
data.to_csv('plots/SF_protection_self_forced.csv', index=False)

In [ ]:
# Figure 4D

bB = 1.0
bW = 2.0
gamma = 1.0

results = np.empty((0, 4))
for file_path in pathlib.Path('results').glob(f'botnet_threshold_SF_N10000_*_bB{bB}_bW{bW}*_g{gamma}*'):

    th = float(file_path.stem.split('k10_th')[1].split('_')[0])
    eps = float(file_path.stem.split('_e')[1].split('_')[0])

    with open(file_path, 'rb') as file:
        data_list = pickle.load(file)

    time = [time for protected, time, total_time in data_list if protected > 0.01]
    time = np.mean(time) if len(time) else 1

    total_time = [total_time for protected, time, total_time in data_list if protected > 0.01]
    total_time = np.mean(total_time) if len(total_time) else 100
    
    results = np.vstack((results,
                         np.array([eps/gamma, th, time, total_time])))

results = results[results[:, 0].argsort()]
results = results[results[:, 1].argsort(kind='mergesort')]

data = pd.DataFrame({'forced_rate': results[:, 0], 'threshold': results[:, 1], 
                     'time': results[:, 2], 'total_time': results[:, 3]})
data.to_csv('plots/SF_botnet_time.csv', index=False)